In [7]:
import re
import math
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
from datetime import timedelta

neg_delta = timedelta(days = 0)
pos_delta = timedelta(days = 2)

df_excel = pd.read_csv('ban.csv', sep = ';', encoding='cp1251')
df_excel = df_excel[pd.notnull(df_excel['Дата исхода заболевания']) & pd.notnull(df_excel['Дата рождения']) & (df_excel['Субъект РФ'] == 'г. Санкт-Петербург')] 
def date_format(column):
    column = column.apply(lambda x: datetime.strptime(x,"%d.%m.%Y"))
    return column
df_excel['Дата исхода заболевания'] = date_format(df_excel['Дата исхода заболевания'])
df_excel['Диагноз установлен'] = date_format(df_excel['Диагноз установлен'])
df_excel['Дата рождения'] = date_format(df_excel['Дата рождения'])
df_excel['Возраст'] = df_excel['Диагноз установлен'] - df_excel['Дата рождения']
df_excel['Возраст'] = df_excel['Возраст'].apply(lambda x: math.floor(x.days / 365.25))

conditions = [
    (df_excel['Возраст'] >= 0) & (df_excel['Возраст'] <= 9),
    (df_excel['Возраст'] >= 10) & (df_excel['Возраст'] <= 19),
    (df_excel['Возраст'] >= 20) & (df_excel['Возраст'] <= 29),
    (df_excel['Возраст'] >= 30) & (df_excel['Возраст'] <= 39),
    (df_excel['Возраст'] >= 40) & (df_excel['Возраст'] <= 49),
    (df_excel['Возраст'] >= 50) & (df_excel['Возраст'] <= 59),
    (df_excel['Возраст'] >= 60) & (df_excel['Возраст'] <= 69),
    (df_excel['Возраст'] >= 70) & (df_excel['Возраст'] <= 79),
    (df_excel['Возраст'] >= 80)
]
cols = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80 и старше']

df_excel['Группа'] = np.select(conditions, cols)

df_fatality = df_excel[pd.notnull(df_excel['Посмертный диагноз'])]
df_fatality = df_fatality[df_fatality['Посмертный диагноз'].str.contains(r'^[JU]')]

df_saved = df_excel[df_excel['Диагноз'].str.contains(r'^[JU]')]
df_saved = df_saved[(df_saved['Исход заболевания'] == 'Выздоровление') |
                    (df_saved['Исход заболевания'] == 'Перевод пациента на амбулаторное лечение')]

df_fatality_1day = df_fatality[(df_fatality['Дата исхода заболевания'] - df_fatality['Диагноз установлен'] < pos_delta) &
                               (df_fatality['Дата исхода заболевания'] - df_fatality['Диагноз установлен'] >= neg_delta)]

def cluster(column):
    final = column.groupby(column.dt.to_period('M')).count()
    return final
df_fatality_1daygb = cluster(df_fatality_1day['Дата исхода заболевания'])
df_fatality_gb = cluster(df_fatality['Дата исхода заболевания'])
df_saved_gb = cluster(df_saved['Дата исхода заболевания'])
df_fatality_age = df_fatality['Возраст'].groupby(df_fatality['Группа']).count()
df_saved_age = df_saved['Возраст'].groupby(df_saved['Группа']).count()

df_fat_final = round((df_fatality_gb / df_saved_gb) * 100, 1)
df_frac_1day = round((df_fatality_1daygb / df_fatality_gb) * 100, 1)
age_group = round((df_fatality_age / df_saved_age) * 100, 2)

df = pd.DataFrame({'Летальность': df_fat_final,
                  'Первые сутки': df_fatality_1daygb.astype('Int64'),
                  'Всего': df_fatality_gb.astype('Int64'),
                  'Доля в первые': df_frac_1day})
df = df.dropna()

cum_saved = df_saved['Дата исхода заболевания'].count()
cum_total = df['Всего'].sum()
cum_fatality = round((cum_total / cum_saved) * 100, 2)
cum_1day = df['Первые сутки'].sum()
cum_frac_1day = round((cum_1day / cum_total) * 100, 2)

df_1 = pd.DataFrame({'A': cum_total,
                  'B': cum_fatality,
                  'C': cum_1day,
                  'D': cum_frac_1day}, index = [0])

writer = pd.ExcelWriter('Книга1.xlsx')
df.to_excel(writer, engine='xlsxwriter', sheet_name='Лист1',startrow=0 , startcol=0)
age_group.to_excel(writer, engine='xlsxwriter', sheet_name='Лист1',startrow=0 , startcol=7)
df_1.to_excel(writer, engine='xlsxwriter', sheet_name='Лист1',startrow=0 , startcol=10)

writer.save()